In [473]:
import numpy as np
import torch
from cspresunet import CSPResUNet
from einops import rearrange
from einops.layers.torch import Rearrange
from torch import nn
from torch.nn import functional as F
from torchsummary import summary

In [474]:
# model = CSPResUNet(1, 1)
# summary(model, input_size=(1, 112, 112))

In [482]:
reshape = Rearrange("(h w) ->h w", h=112, w=112)
reshape_bk = Rearrange("(h w) b c -> b c h w", h=112, w=112)
hwcc2nc2 = Rearrange("h w c1 c2 -> (h w) (c1 c2)")
total = 112 * 112
m = reshape(torch.arange(total))
adj = torch.zeros(total, total)
emb_dim = 2

In [476]:
neighbor_m = F.pad(m, (1, 1, 1, 1), value=-1).unfold(0, 3, 1).unfold(1, 3, 1)
lookup_tb = hwcc2nc2(neighbor_m)
for i in range(112 * 112):
    result = lookup_tb[i]
    neighbors = result[(result != -1)]
    adj[i, neighbors] = True
adj = adj.bool()
W = nn.Linear(8, emb_dim)
a = nn.Linear(2 * emb_dim, 1)

In [477]:
o = torch.rand(16, 8, 112, 112)
b, c, h, w = o.shape
o = rearrange(o, "b c h w -> b (h w) c")
output = torch.zeros((h * w, b, emb_dim))

In [478]:
for i in torch.arange(total):
    n = adj[i].sum()
    neighbor_idx = torch.where(adj[i] == True)[0]
    ref_node = torch.index_select(o, 1, i).repeat_interleave(n, dim=1)
    nei_nodes = torch.index_select(o, 1, neighbor_idx)
    ref_h_prime = W(ref_node)
    nei_h_prime = W(nei_nodes)
    stack = torch.cat([ref_h_prime, nei_h_prime], dim=2)
    activated = F.leaky_relu(a(stack))
    s = torch.sum(activated, dim=1, keepdim=True)
    alpha = F.softmax(activated / s, dim=1)
    output[i] = (alpha * nei_h_prime).sum(1)

In [479]:
output.shape

torch.Size([12544, 16, 2])

In [481]:
reshape_bk(output).shape

torch.Size([16, 2, 112, 112])